## Analyze whether TOBS varies more from year to year or from place to place.

In [1]:
%config InlineBackend.figure_format = 'retina'
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext, SparkConf
#sc.stop()
conf = SparkConf()
conf = conf.setAppName("temp1")
conf = conf.setMaster("local")
conf = conf.set("spark.driver.host", "localhost")
sc = SparkContext(conf=conf, master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SSSSSSBB'
meas='TOBS'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
1271
+------------------+------------------+-------------------+---------+--------+--------+---------+-----------+------------------+-------------------+-------------------+-------------------+-----------+-----------+------+--------------------+------+
|           coeff_1|           coeff_2|            coeff_3|elevation|   label|latitude|longitude|measurement|             res_1|              res_2|              res_3|           res_mean|    station|  total_var|undefs|              vector|  year|
+------------------+------------------+-------------------+---------+--------+--------+---------+-----------+------------------+-------------------+-------------------+-------------------+-----------+-----------+------+--------------------+------+
|-835.8790377246141|265.03257181517876|  -309.765316879443|    352.0|SSSSSSBB| 33.3814|  -112.07|       TOBS|0.5038199540752998| 0.3887194464613411|  0.343293660417636|0.04729194803

In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [19]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
          'res_1', 'res_2', 'res_3', 'res_mean', 'station','year'])
#Features='station, year, coeff_2'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT coeff_1, coeff_2, coeff_3, elevation, latitude, longitude, res_1, res_2, res_3, res_mean, station, year FROM weather


,coeff_1,coeff_2,coeff_3,elevation,latitude,longitude,res_1,res_2,res_3,res_mean,station,year
0,-835.879038,265.032572,-309.765317,352.0,33.3814,-112.0700,0.503820,0.388719,0.343294,0.047292,USC00028112,2004.0
1,844.472626,-465.249760,-12.309730,417.6,33.7044,-115.6289,0.496912,0.344210,0.344103,0.105363,USC00043855,1951.0
2,840.888373,-339.180027,157.341710,381.0,33.5000,-111.9833,0.459190,0.368859,0.345625,0.090670,USC00020406,1965.0
3,996.559076,-334.206856,-141.854011,376.4,33.4114,-111.8183,0.421093,0.361197,0.348638,0.123223,USC00025467,1971.0
4,1139.914659,-417.231045,16.229481,355.7,33.4258,-111.9217,0.450631,0.348744,0.348669,0.102705,USC00028499,1953.0


In [13]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_2')
year_station_table.head(10)

station,USC00020060,USC00020104,USC00020406,USC00020949,USC00021026,USC00021161,USC00021282,USC00021353,USC00021356,USC00021361,...,USC00029634,USC00040924,USC00042598,USC00043855,USC00044259,USC00045502,USC00045860,USC00046635,USW00023183,USW00093140
year,,,,,,,,,,,,,,,,,,,,,
1901.0,NaN,NaN,NaN,NaN,-682.192311,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1902.0,NaN,NaN,NaN,NaN,-975.685812,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1903.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1904.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1905.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1906.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1907.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-431.460137,NaN,NaN,NaN,NaN,NaN
1908.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1909.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the MS before and after subtracting either  the row or the column vector.

In [51]:
def MS(Mat):
    return np.nanmean(Mat**2)

def MS_year_station(coeff='coeff_1'):
    year_station_table=pdf.pivot(index='year', columns='station', values=coeff)
    mean_by_year=np.nanmean(year_station_table,axis=1)
    mean_by_station=np.nanmean(year_station_table,axis=0)
    tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
    tbl_minus_station = year_station_table-mean_by_station

    print 'total MS                    = %.3f' % MS(year_station_table)
    print 'MS removing mean-by-station = %.3f, fraction explained = %.1f%%' % \
        (MS(tbl_minus_station), 100 - MS(tbl_minus_station) * 100.0 / MS(year_station_table))
    print 'MS removing mean-by-year    = %.3f, fraction explained = %.1f%%' % \
        (MS(tbl_minus_year), 100 - MS(tbl_minus_year) * 100.0 / MS(year_station_table))

In [52]:
MS_year_station(coeff='coeff_1')

total MS                    = 498922.307
MS removing mean-by-station = 239829.991, fraction explained = 51.9%
MS removing mean-by-year    = 420598.281, fraction explained = 15.7%


In [53]:
MS_year_station(coeff='coeff_2')

total MS                    = 250027.553
MS removing mean-by-station = 79785.341, fraction explained = 68.1%
MS removing mean-by-year    = 123157.961, fraction explained = 50.7%


In [54]:
MS_year_station(coeff='coeff_3')

total MS                    = 34814.772
MS removing mean-by-station = 31268.286, fraction explained = 10.2%
MS removing mean-by-year    = 15841.983, fraction explained = 54.5%


In [36]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 186.587170803
0 after removing mean by year    = 125.864941319
0 after removing mean by stations= 113.310557224
1 after removing mean by year    = 112.642086338
1 after removing mean by stations= 112.558595556
2 after removing mean by year    = 112.543023921
2 after removing mean by stations= 112.53950424
3 after removing mean by year    = 112.538627704
3 after removing mean by stations= 112.538395997
4 after removing mean by year    = 112.538331875
4 after removing mean by stations= 112.538313376
